In [380]:
from numpy.testing import assert_array_equal
import numpy as np
from typing import Tuple
from nptyping import Array
import unittest
import itertools


def bat2int(bit_arr: Tuple[bool]) -> int:
    bit_arr_str = ''.join(str(int(b)) for b in bit_arr)
    return int(bit_arr_str, 2)


def ba2int(bit_arr: Array[bool, 1, ...]) -> int:
    return bat2int(bit_arr)


def int2bastr(i: int, digit: int) -> str:
    return f'{i:0{digit}b}'


def bastr2bat(bit_arr_str: str) -> Tuple[bool]:
    return tuple(bool(int(s)) for s in bit_arr_str)


def bastr2ba(bit_arr_str: str) -> Array[bool, 1, ...]:
    return np.array(bastr2bat(bit_arr_str))


def int2bat(i: int, digit: int) -> Tuple[bool]:
    return bastr2bat(int2bastr(i, digit))


def int2ba(i: int, digit: int) -> Array[bool, 1, ...]:
    return bastr2ba(int2bastr(i, digit))


def gen_all_bool_patterns(length: int) -> Tuple[bool]:
    """Return all bool patterns. Be aware of the order of patterns.

    Arguments:
        length {int} -- bit length

    Returns:
        Tuple[Tuple[bool]] -- bool patterns (e.g, ((False, False), (False, True),
        (True, False), (True, True)))
    """
    tpls = tuple(((False, True)) for _ in range(length))
    return tuple(itertools.product(*tpls))


def all_assert_equal(test_case: unittest.TestCase, expecteds: Tuple, actuals: Tuple):
    """Exec assert equal for passed test patterns.

    Arguments:
        test_case {unittest.TestCase} -- TestUnit instance to exec assertEqual
        expecteds {Tuple} -- expected sequence
        actuals {Tuple} -- actual sequence
    """
    for e, a in zip(expecteds, actuals):
        test_case.assertEqual(e, a)


def all_assert_array_equal(expecteds: Tuple[Array[bool]], actuals: Tuple[Array[bool]]):
    for e, a in zip(expecteds, actuals):
        assert_array_equal(e, a)


In [381]:
import numpy as np
from nptyping import Array
from typing import Callable
import functools


def NOT(x: bool) -> bool:
    return not x


def _AND(a: bool, b: bool) -> bool:
    return a and b


def AND(*xs: bool) -> bool:
    return functools.reduce(_AND, xs)


def _OR(a: bool, b: bool) -> bool:
    return a or b


def OR(*xs: bool) -> bool:
    return functools.reduce(_OR, xs)


def NAND(*xs: bool) -> bool:
    return NOT(AND(*xs))


def NOR(*xs: bool) -> bool:
    return NOT(OR(*xs))


def _XOR(a: bool, b: bool) -> bool:
    return OR(AND(a, NOT(b)), AND(NOT(a), b))


def XOR(*xs: bool) -> bool:
    return functools.reduce(_XOR, xs)


def HA(a: bool, b: bool) -> Array[bool, 1, 2]:
    """Half Adder

    Arguments:
        a {bool} -- operand a
        b {bool} -- operand b

    Returns:
        Tuple[bool, bool] -- carry, sum
    """
    c = AND(a, b)
    s = XOR(a, b)
    return np.array((c, s))


def FA(cin: bool, a: bool, b: bool) -> Array[bool, 1, 2]:
    """Full Adder

    Arguments:
        cin {bool} -- input carry
        a {bool} -- operand a
        b {bool} -- operand b

    Returns:
        Array[bool, 1, 2] -- carry, sum
    """
    t1_c, t1_s = HA(a, b)
    t2_c, t2_s = HA(cin, t1_s)
    c = OR(t2_c, t1_c)
    return np.array((c, t2_s))


def ALU(cin: bool, arr_a: Array[bool, 1, 4], arr_b: Array[bool, 1, 4]) \
        -> Array[bool, 1, 5]:
    """ALU: 4-bit Full Adder

    Arguments:
        cin {bool} -- input carry
        arr_a {Array[bool, 1, 4]} -- 4-bit array as operand a
        arr_b {Array[bool, 1, 4]} -- 4-bit array as operand b

    Raises:
        ValueError: Length of arr_a and arr_b must be 4

    Returns:
        Array[bool, 1, 5] -- 0th bit is carry, others are sums (LSB is index=1, MSB is index=5)
    """
    if arr_a is None or arr_b is None or len(arr_a) != 4 or len(arr_b) != 4:
        raise ValueError('Length of each input operands must be 4')

    c = cin

    def _FA():
        nonlocal c
        for a, b in zip(arr_a, arr_b):
            c, s = FA(c, a, b)
            yield s

    sums = tuple(_FA())
    return np.array((c,) + sums)


def AR(a: bool, b: bool, c: bool, d: bool, g1_: bool, g2_: bool) -> Array[bool, 1, 16]:
    """Address Resolver. Convert 4-bit signal to one of 16 address for ROM.
    e.g, Returned (True, False, ..., False) implies 0th address in ROM.

    Arguments:
        a {bool} -- 0th bit
        b {bool} -- 1st bit
        c {bool} -- 2nd bit
        d {bool} -- 3rd bit
        g1 {bool} -- must be False
        g2 {bool} -- must be False

    Returns:
        Array[bool, 1, 16] -- Signal to spesify address of ROM (LSB is index=0)
    """
    g = NOT(NAND(NOT(g1_), NOT(g2_)))
    t0 = NOT(NAND(NOT(a), NOT(b)))
    t1 = NOT(NAND(a, NOT(b)))
    t2 = NOT(NAND(NOT(a), b))
    t3 = NOT(NAND(a, b))
    t4 = NOT(NAND(NOT(c), NOT(d)))
    t5 = NOT(NAND(c, NOT(d)))
    t6 = NOT(NAND(NOT(c), d))
    t7 = NOT(NAND(c, d))
    return np.array([NOT(NAND(g, i, j)) for i in (t4, t5, t6, t7) for j in (t0, t1, t2, t3)])


def _MUX(a: bool, b: bool, c0: bool, c1: bool, c2: bool, c3: bool) -> bool:
    t0 = AND(c0, NOT(a), NOT(b))
    t1 = AND(c1, a, NOT(b))
    t2 = AND(c2, NOT(a), b)
    t3 = AND(c3, a, b)
    return OR(t0, t1, t2, t3)


def MUX(a: bool, b: bool,
        ca: Array[bool, 1, 4], cb: Array[bool, 1, 4], cc: Array[bool, 1, 4], cd: Array[bool, 1, 4])\
        -> Array[bool, 1, 4]:
    """4-input Multiplexer

    Arguments:
        a {bool} -- select a
        b {bool} -- select b
        ca {Array[bool, 1, 4]} -- 1st input 4-bit array
        cb {Array[bool, 1, 4]} -- 2nd input 4-bit array
        cc {Array[bool, 1, 4]} -- 3rd input 4-bit array
        cd {Array[bool, 1, 4]} -- 4th input 4-bit array

    Returns:
        Array[bool, 1, 4] -- Selected 4-bit array
    """
    return np.array(tuple(_MUX(a, b, ca[i], cb[i], cc[i], cd[i]) for i in range(4)))


def DECODER(op_arr: Array[bool, 1, 4], c_flag_: bool) -> Array[bool, 1, 6]:
    """Instruction Decoder

    Arguments:
        op_arr {Array[bool, 1, 4]} -- 4-bit operation code
        c_flag_ {bool} -- negative carry flag

    Returns:
        Array[bool, 1, 6] -- [select_a, select_b, load0_, load1_, load2_, load3_]
    """
    op0, op1, op2, op3 = op_arr
    select_a = OR(op0, op3)
    select_b = op1
    load0_ = OR(op2, op3)
    load1_ = OR(NOT(op2), op3)
    load2_ = NAND(NOT(op2), op3)
    load3_ = NAND(op2, op3, OR(op0, c_flag_))
    return np.array((select_a, select_b, load0_, load1_, load2_, load3_))

In [ ]:
def make_REGISTER(ent: bool, enp: bool) -> Callable(Array[bool, 1, 4]):
    def COUNTER(load_: bool, state: Array[bool, 1, 4]) -> Array[bool, 1, 4]:
        _state = state
        while True:
            input_arr = yield _state
            state, _state = _state, state
            if load_ is False:
                state = input_arr
            else:
                res = ALU(_state, bastr2ba('0001'))
                state = res[1:]
    
    def REGISTER(load_: bool, state: Array[bool, 1, 4]) -> Array[bool, 1, 4]:
        _state = state
        while True:
            input_arr = yield _state
            if load_ is False:
                state, _state = _state, state
                state = input_arr
    
    if ent and enp:
        return COUNTER(False, )
    if (ent is False) and (enp is False):
        return REGISTER(1, )

In [49]:
import numpy as np
a = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
a

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [50]:
b = np.array([False, True, False])
b

array([False,  True, False])

In [53]:
a[b][0]

array([3, 4, 5])

In [52]:
a[0]

array([0, 1, 2])

In [54]:
load_ = False
if load_ is False:
    print('False')
else:
    print('True')

False


In [67]:
arr = np.array([False, True, False])
a, b, c = arr

In [75]:
t = (False,) * 3
t

(False, False, False)

In [76]:
t[1] = True
t

TypeError: 'tuple' object does not support item assignment

In [81]:
from tqdm import trange
import time

for i in trange(100):
    time.sleep(0.01)

100%|██████████| 100/100 [00:01<00:00, 87.37it/s]


In [1]:
from enum import Enum

class ClockCycle(Enum):
    NORMAL = 1
    HIGH = 10
    MANUAL = 0

In [6]:
for i, cc in enumerate(ClockCycle):
    print(cc.value)

1
10
0


In [7]:
len(ClockCycle)

3

In [12]:
cn = ClockCycle.NORMAL
cn

<ClockCycle.NORMAL: 1>

In [13]:
cn is ClockCycle.NORMAL

True

In [92]:
for i, (name, member) in enumerate(ClockCycle.__members__.items()):
    print(i, name, member)

0 NORMAL ClockCycle.NORMAL
1 HIGH ClockCycle.HIGH
2 MANUAL ClockCycle.MANUAL


In [94]:
clock_cycles = tuple(ClockCycle.__members__.items())
clock_cycles

(('NORMAL', <ClockCycle.NORMAL: 1>),
 ('HIGH', <ClockCycle.HIGH: 10>),
 ('MANUAL', <ClockCycle.MANUAL: 0>))

In [113]:
input_int = -1
while not (0 <= input_int < len(clock_cycles)):
    menu_str = 'RUN: '
    for i, cc in enumerate(clock_cycles):
        menu_str += f'[{i}] {cc[0]}'
        menu_str += f'({cc[1].value}Hz)   ' if cc[1].value > 0 else '  '
    menu_str += '> '
    print()

'RUN: [0] NORMAL(1Hz)   [1] HIGH(10Hz)   [2] MANUAL  > '

In [117]:
import re
length = len(clock_cycles)
menu_num_pattern = re.compile(r'^[0-' f'{length}]$')
menu_num_pattern

re.compile(r'^[0-3]$', re.UNICODE)

In [119]:
menu_num_pattern.fullmatch('1')

<re.Match object; span=(0, 1), match='1'>

In [21]:
def DISPLAY(**kwargs):
    print('first:{first[::-1]},,,second:{second}'.format(**kwargs))

In [22]:
DISPLAY(**{'first': [0, 1, 2], 'second': 2})

TypeError: list indices must be integers or slices, not str

In [23]:
1 / 10

0.1